In [1]:
import numpy as np
from scipy.spatial import ConvexHull
import tifffile as tiff
from numba import njit
import os
import pandas as pd
from skimage.filters import gaussian

In [2]:
def readfile(csv_file):
    # Read the CSV file into a DataFrame
    df = pd.read_csv(csv_file)

    cell_points = {}

    # Iterate through the DataFrame and store the cell index at the corresponding x, y, z position in the output array
    cell_index_set = set([int(_) for _ in df['Cell Index']])
    for index in cell_index_set:
        cell_points[index + 1] = []

    xmax, ymax, zmax = 0, 0, 0
    for index, row in df.iterrows():
        x, y, z = int(row['x_in_pix']), int(row['y_in_pix']), int(row['z_in_pix'])
        cell_index = int(row['Cell Index']) + 1
        cell_points[cell_index].append([x, y, z])
        xmax, ymax, zmax = max(x, xmax), max(y, ymax), max(z, zmax)
    min_output_shape = (xmax, ymax, zmax)

    return cell_points, min_output_shape


@njit
def get_filled_hull_numba(mask, equations, scan_range):
    for z in range(scan_range[0][2], scan_range[1][2] + 1):
        for y in range(scan_range[0][1], scan_range[1][1] + 1):
            for x in range(scan_range[0][0], scan_range[1][0] + 1):
                inside_hull = True
                for i in range(equations.shape[0]):
                    plane = equations[i]
                    if plane[0] * x + plane[1] * y + plane[2] * z + plane[3] > 0:
                        inside_hull = False
                        break
                if inside_hull:
                    mask[x, y, z] = True


def get_filled_hull(hull, shape, scan_range):
    mask = np.zeros(shape, dtype=bool)
    get_filled_hull_numba(mask, hull.equations, scan_range)
    return mask


def generate_labeled_tif(input_data, output_shape = (2068, 2068, 158)):
    labels = list(input_data.keys())
    filled_hull = np.zeros(output_shape, dtype=np.uint16)
    smoothed_filled_hull = np.zeros(output_shape, dtype=np.uint16)

    for label in labels:
        if len(input_data[label]) <= 10:
            continue
        points = np.array(input_data[label])
        scan_range = np.array([np.min(points, axis=0), np.max(points, axis=0)])

        hull = ConvexHull(points)
        mask = get_filled_hull(hull, output_shape, scan_range)
        filled_hull[mask] = label
        
        # # gaussian blur
        # tmp_filled_hull = np.zeros(output_shape, dtype=np.float32)
        # tmp_filled_hull[mask] = 255
        # smoothed_hull = gaussian(filled_hull, sigma=7, preserve_range=True)
        
        # # binary
        # binary_hull = (smoothed_hull > 127).astype(np.uint16)
        
        # # gray color
        # colored_hull = binary_hull * label

        # # add current hull to tif file
        # smoothed_filled_hull += colored_hull

    return filled_hull, smoothed_filled_hull

In [3]:
# os.chdir('./cell_segment_show/')
input_filename = 'rna_labeled.csv'
output_filename = 'hull_cells'

In [4]:
input_cells, min_output_shape = readfile(csv_file = f'./{input_filename}')

In [7]:
sub_input_cells = {}
for index in range(1000, 1010):
    sub_input_cells[index] = input_cells[index]

In [8]:
filled_hull, smoothed_filled_hull = generate_labeled_tif(sub_input_cells, output_shape = min_output_shape)

tiff.imwrite(f'./hull_results/{output_filename}.tif', np.transpose(filled_hull, (2, 1, 0)))
# tiff.imwrite(f'./hull_results/{output_filename}_smoothed.tif', np.transpose(smoothed_filled_hull, (2, 1, 0)))